In [2]:
!pip3 install torch==1.7.1
!pip3 install tfrecord

  Using cached https://files.pythonhosted.org/packages/90/4f/acf48b3a18a8f9223c6616647f0a011a5713a985336088d7c76f3a211374/torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/fd/247ef25f5ec5f9acecfbc98ca3c6aaf66716cf52509aca9a93583d410493/protobuf-3.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Usin

In [3]:
import json
import torch
from tfrecord.torch.dataset import TFRecordDataset
import tensorflow as tf

In [4]:
path_data = "/data/jcabrera/didi_wo_text/training/"
name_file = "diagrams_wo_text_20200131-00000-of-00010"

In [5]:
tfrecord_path = path_data + name_file
index_path = None

In [6]:
description = {"key": "byte", "label_id":"byte",
               "ink":"float", "stroke_length":"int", "shape":"int", "num_strokes":"int", 
               "rdp_ink":"float", "rdp_stroke_length":"int", 
               "rdp_shape":"int", "rdp_num_strokes":"int",
               }

In [8]:
dataset = TFRecordDataset(tfrecord_path, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [9]:
dataiter = iter(loader)
data = dataiter.next()
print(data)

{'key': tensor([[ 48,  48,  51,  48,  55,  56,  57,  97,  57,  54,  97,  97, 101,  55,
         101,  97]], dtype=torch.uint8), 'label_id': tensor([[101,  50,  51,  56,  56,  97, 100,  56,  55,  98, 102,  55,  97,  52,
          98,  54,  49,  97,  51,  99,  52,  97,  55,  52, 102,  51,  49,  53,
          48, 101,  51, 100,  98,  97,  57,  56,  49, 100,  49,  97]],
       dtype=torch.uint8), 'ink': tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[ 32,  32,  25,  43, 128,  31,  33,  34,  29]], dtype=torch.int32), 'shape': tensor([[  9, 128,   4]], dtype=torch.int32), 'num_strokes': tensor([[9]], dtype=torch.int32), 'rdp_ink': tensor([[187.4000, 751.9100,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[38, 36, 33, 21, 29, 19, 16, 13, 16]], dtype=torch.int32), 'rdp_shape': tensor([[ 9, 38,  4]], dtype=torch.int32), 'rdp_num_strokes': tensor([[9]], dtype=torch.int32)}


/home/shuaman/.local/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])


In [10]:
data["ink"]

tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [11]:
data["shape"]

tensor([[  9, 128,   4]], dtype=torch.int32)

In [12]:
data2 = dataiter.next()
print(data2)

{'key': tensor([[ 48,  48,  51,  97,  52,  53,  56,  97,  54,  48,  55,  51,  98,  98,
         102, 100]], dtype=torch.uint8), 'label_id': tensor([[100,  56,  50, 101,  99,  52,  48,  99,  99,  98, 102,  98,  98,  49,
          49,  57,  56,  99, 102,  49, 100,  52,  52,  48,  57, 102,  57,  51,
          57, 102,  52,  98,  49,  56, 102,  53,  50,  98, 100,  55]],
       dtype=torch.uint8), 'ink': tensor([[0.0699, 0.0804, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[194,  10,   9,  29,  25,  19,  11,   9, 202,   2,  11,  11,  99,  37,
          22,  22,  33,  31, 108,  61,  78,  26,  47,  25,  18,  10,  11,  41,
         188,   9,   9,  31,  49,  30,  31,  39]], dtype=torch.int32), 'shape': tensor([[ 36, 202,   4]], dtype=torch.int32), 'num_strokes': tensor([[36]], dtype=torch.int32), 'rdp_ink': tensor([[584.8800, 192.7900,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[30,  4,  4,  5,  4,  3,  6,  3, 13,  2,  4,  6,  6,  6,  6,  

In [13]:
data2["ink"].shape

torch.Size([1, 29088])